In [1]:
# Start Python Imports 
import time
start_time = time.time()

import math, random, datetime
# Data Manipulation 
import numpy as np 
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt 
import missingno 
import seaborn as sns 
plt.style.use('seaborn-whitegrid')

In [16]:
df_70 = pd.read_csv('../final_data/F70_train.csv')
list_70_features = list(df_70.columns)
list_70_features.remove('Response')

df_40 = pd.read_csv('../final_data/F40.csv')
list_40_features = list(df_40.columns)
list_40_features.remove('Response')

In [21]:
from sklearn.preprocessing import LabelEncoder

# prepare out test sets for prediction
df_hold_out = pd.read_csv('../final_data/F70_test.csv') 

#df_X_test_hold_out_70 = df_hold_out[list_70_features]
#df_X_test_hold_out_40 = df_hold_out[list_40_features]

# this will be used to get actual/real performance on holdout set
df_y_test_hold_out = df_hold_out[['Response']]

print('hold out set processed')


# prepare raw_test_70 and raw_test_40
df_raw_test = pd.read_csv('../raw_data/test.csv')

# 1. intruduce medical_keyword_sum 
# 2. encode product_info_2
# 3. impute
# 4. Only take features that we need..this way we don't have to do selective drop of features

df_raw_test['Medical_Keyword_SUM'] = df_raw_test.loc[:, 'Medical_Keyword_1':'Medical_Keyword_48'].sum(axis = 1, skipna = True)


encoder = LabelEncoder()
Product_Info_2 = df_raw_test['Product_Info_2']
Product_Info_2_encoded = encoder.fit_transform(Product_Info_2)

# let's add this array to dataframe...
df_raw_test['Product_Info_2_encoded'] = Product_Info_2_encoded

# Impute..categorical variables with mode...cont var with mean
df_raw_test['Medical_History_1'].fillna(df_raw_test['Medical_History_1'].mode()[0], inplace=True)
df_raw_test.fillna(df_raw_test.mean(), inplace=True)

#df_X_test_raw_70 = df_raw_test[list_70_features]
#df_X_test_raw_40 = df_raw_test[list_40_features]

print('raw test set processed')

dict_test_sets = {'hold_out_70':df_hold_out, 'hold_out_40':df_hold_out,\
                  'raw_test_70':df_raw_test, 'raw_test_40':df_raw_test}

print('test sets have been put into dictionary')

hold out set processed
raw test set processed
test sets have been put into dictionary


In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

# create best models from our analysis
model_log = LogisticRegression(C=10000.0, penalty='l2', random_state=0)
model_svc = SVC(C=2.782559402207126, cache_size=200, class_weight=None, coef0=0.0,\
    decision_function_shape='ovr', degree=3, gamma=0.005623413251903491,\
    kernel='rbf', max_iter=-1, probability=False, random_state=0,\
    shrinking=True, tol=0.001, verbose=False)
dict_models = {'logistic': model_log, 'SVC': model_svc}

# we have 4 training sets - F70, F40, B70, B40 for each model and 4 test(2 from holdout and 2 from raw test sets)
dict_training_sets = {'F70':pd.read_csv('../final_data/F70_train.csv'), 'F40':pd.read_csv('../final_data/F40.csv'), \
                      'B70':pd.read_csv('../final_data/B70.csv'), 'B40':pd.read_csv('../final_data/B40.csv')}


# we will use values from dictionaries to process...and log the keys into list of dictionaries...
# later we will convert list_all_results to a dataframe to get a complete overview
list_all_results = []

for m in dict_models:
    scaled = m == 'SVC'
    model = dict_models[m]
    for train in dict_training_sets:
        result = {}
        result['model'] = m
        result['training_set'] = train
        features = list(dict_training_sets[train].columns)
        features.remove('Response')
        num_features = '70' if '70' in train else '40'
        train_X = dict_training_sets[train][features]
        if scaled:
            train_X = preprocessing.scale(train_X)
        train_y = dict_training_sets[train][['Response']]
        model.fit(train_X, train_y)
        score = cross_val_score(model, train_X, train_y, cv=StratifiedKFold(3), n_jobs=3)
        result['cross_val_score'] = sum(score) / len(score)
        for test in dict_test_sets:
            if num_features not in test:
                continue
            #result['test_set'] = test
            test_X = dict_test_sets[test][features]
            if scaled:
                test_X = preprocessing.scale(test_X)
            pred_result = model.predict(test_X)
            output = pd.concat([dict_test_sets[test], pd.DataFrame({'Response':pred_result})], axis=1, sort=False)
            fileName = f'{m}_{train}_{test}_result.csv'
            output.to_csv(f'../partial_data/{fileName}', index=False)
            #result['generated_file'] = fileName
        list_all_results.append(result)


/home/ubuntu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ubuntu/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ubuntu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ubuntu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ubuntu/.local/lib/python3.6/site-packages/sklearn/utils/

In [36]:
# convert list_all_results to a dataframe to get a complete overview
pd.DataFrame(list_all_results).head()

,model,training_set,cross_val_score
0,logistic,F70,0.466024
1,logistic,F40,0.462908
2,logistic,B70,0.369290
3,logistic,B40,0.367747
4,SVC,F70,0.492443


In [29]:
from sklearn.metrics import accuracy_score

# check our performance on hold-out sets...
#test_data = pd.read_csv(f'../final_data/F70_test.csv')
#y_test = test_data[['Response']]

filenames = ['logistic_B40_hold_out_40_result.csv',
            'logistic_B70_hold_out_70_result.csv',
            'logistic_F40_hold_out_40_result.csv',
            'logistic_F70_hold_out_70_result.csv',
            'SVC_B40_hold_out_40_result.csv',
            'SVC_B70_hold_out_70_result.csv',
            'SVC_F40_hold_out_40_result.csv',
            'SVC_F70_hold_out_70_result.csv']
for filename in filenames:
    if 'hold_out' not in filename:
        continue
    result_data = pd.read_csv(f'../partial_data/{filename}')

    y_test = result_data[['Response']]
    y_result = result_data.iloc[:,-1]

    score = accuracy_score(y_test, y_result)
    print(f'Score of {filename}: {score}')

Score of logistic_B40_hold_out_40_result.csv: 0.39302854256125286
Score of logistic_B70_hold_out_70_result.csv: 0.39353372063652436
Score of logistic_F40_hold_out_40_result.csv: 0.4643428475204176
Score of logistic_F70_hold_out_70_result.csv: 0.46703713058853247
Score of SVC_B40_hold_out_40_result.csv: 0.3587606297886672
Score of SVC_B70_hold_out_70_result.csv: 0.3583396480592742
Score of SVC_F40_hold_out_40_result.csv: 0.49355897954028793
Score of SVC_F70_hold_out_70_result.csv: 0.4954112991496169


In [32]:
# based on the cross val scores and performance on hold-out sets, we choose to submit....
test_data = pd.read_csv(f'../raw_data/test.csv')
result_data = pd.read_csv(f'../partial_data/logistic_F70_raw_test_70_result.csv')

output = pd.concat([test_data[['Id']], result_data[['Response']]], axis=1, sort=False)
output.to_csv('../results/submission_lr.csv', index=False)